In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import os
import random
import shutil

In [2]:
# Set random seed for reproducibility
random.seed(42)
tf.random.set_seed(42)

# Function to create train, validation, and test sets
def create_dataset_splits(source_dir, train_dir, val_dir, test_dir, num_train, num_val, num_test):
    for dir in [train_dir, val_dir, test_dir]:
        os.makedirs(os.path.join(dir, 'cats'), exist_ok=True)
        os.makedirs(os.path.join(dir, 'dogs'), exist_ok=True)

    for animal in ['cat', 'dog']:
        animal_files = [f for f in os.listdir(source_dir) if f.startswith(animal)]
        random.shuffle(animal_files)

        for i, file in enumerate(animal_files):
            if i < num_train:
                shutil.copy(os.path.join(source_dir, file), os.path.join(train_dir, f'{animal}s', file))
            elif i < num_train + num_val:
                shutil.copy(os.path.join(source_dir, file), os.path.join(val_dir, f'{animal}s', file))
            elif i < num_train + num_val + num_test:
                shutil.copy(os.path.join(source_dir, file), os.path.join(test_dir, f'{animal}s', file))
            else:
                break

    print(f"Train set: {len(os.listdir(os.path.join(train_dir, 'cats'))) + len(os.listdir(os.path.join(train_dir, 'dogs')))} images")
    print(f"Validation set: {len(os.listdir(os.path.join(val_dir, 'cats'))) + len(os.listdir(os.path.join(val_dir, 'dogs')))} images")
    print(f"Test set: {len(os.listdir(os.path.join(test_dir, 'cats'))) + len(os.listdir(os.path.join(test_dir, 'dogs')))} images")


# Create dataset splits
source_dir = 'D:/DeepLearning/Assignment2/dogs-vs-cats/train'
train_dir = 'D:/DeepLearning/Assignment2/dataset/train'
val_dir = 'D:/DeepLearning/Assignment2/dataset/validation'
test_dir = 'D:/DeepLearning/Assignment2/dataset/test'

create_dataset_splits(source_dir, train_dir, val_dir, test_dir, 1000, 500, 500)


Train set: 2000 images
Validation set: 1000 images
Test set: 1000 images


In [3]:
# Data preprocessing
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2,
                                   height_shift_range=0.2, shear_range=0.2, zoom_range=0.2,
                                   horizontal_flip=True, fill_mode='nearest')

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(150, 150),
                                                    batch_size=32, class_mode='binary')

validation_generator = val_test_datagen.flow_from_directory(val_dir, target_size=(150, 150),
                                                            batch_size=32, class_mode='binary')

test_generator = val_test_datagen.flow_from_directory(test_dir, target_size=(150, 150),
                                                      batch_size=32, class_mode='binary')


Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:

# Model definition
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    # Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.keras', monitor='val_accuracy', save_best_only=True, mode='max')

# Train the model
history = model.fit(train_generator, epochs=30, validation_data=validation_generator,
                    callbacks=[early_stopping, model_checkpoint])

C:\Users\dell\anaconda3\envs\test_env\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\dell\anaconda3\envs\test_env\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5209 - loss: 0.9183

C:\Users\dell\anaconda3\envs\test_env\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


63/63 ━━━━━━━━━━━━━━━━━━━━ 105s 2s/step - accuracy: 0.5210 - loss: 0.9158 - val_accuracy: 0.5950 - val_loss: 0.6806
Epoch 2/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 81s 1s/step - accuracy: 0.5737 - loss: 0.6761 - val_accuracy: 0.6050 - val_loss: 0.6707
Epoch 3/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 77s 1s/step - accuracy: 0.5613 - loss: 0.6738 - val_accuracy: 0.5640 - val_loss: 0.6929
Epoch 4/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 78s 1s/step - accuracy: 0.5887 - loss: 0.6755 - val_accuracy: 0.5980 - val_loss: 0.6751
Epoch 5/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 84s 1s/step - accuracy: 0.6015 - loss: 0.6680 - val_accuracy: 0.6260 - val_loss: 0.6612
Epoch 6/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 105s 2s/step - accuracy: 0.6231 - loss: 0.6557 - val_accuracy: 0.6350 - val_loss: 0.6538
Epoch 7/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 80s 1s/step - accuracy: 0.6420 - loss: 0.6354 - val_accuracy: 0.6240 - val_loss: 0.6503
Epoch 8/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 84s 1s/step - accuracy: 0.6313 - loss: 0.6467 - val_accuracy: 0.6570 - val_loss: 0.6284
E

In [ ]:

# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

# Find the best epoch
best_epoch = np.argmax(history.history['val_accuracy']) + 1
print(f"Best epoch: {best_epoch}")
print(f"Best validation accuracy: {max(history.history['val_accuracy']):.4f}")

# Evaluate on test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test accuracy: {test_accuracy:.4f}")